In [13]:
import requests
import pandas as pd
from datetime import datetime

In [14]:
SERVERS = [
    {"name": "Server1", "url": "http://127.0.0.1:8001", "type": "temperature"},
    {"name": "Server2", "url": "http://127.0.0.1:8002", "type": "temperature"},
    {"name": "PDU1", "url": "http://127.0.0.1:9001", "type": "power"}
]

# Redfish のエンドポイント設定
ENDPOINTS = {
    "temperature": "/redfish/v1/Chassis/1U/Thermal",
    "power": "/redfish/v1/Chassis/0/Power"
}

# 取得開始時刻（ファイル名用）
START_TIME = datetime.now().strftime("%Y%m%d_%H%M%S")

# 出力CSVファイル名
CSV_FILE = f"server_pdu_data_{START_TIME}.csv"

In [40]:
def get_redfish_data(server):
    """指定した Redfish API サーバー（サーバー温度 / PDU 電力）からデータを取得"""
    url = f"{server['url']}{ENDPOINTS[server['type']]}"
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    try:
        response = requests.get(url, verify=False, timeout=1)  # SSL無視（Mockup Server用）、タイムアウト5秒
        response.raise_for_status()
        data = response.json()

        results = []

        if server["type"] == "temperature":
            # CPU 温度データを抽出
            for sensor in data.get("Temperatures", []):
                if "CPU" in sensor.get("Name", ""):
                    results.append({
                        "Timestamp": timestamp,
                        "Device": server["name"],
                        "IP":server["url"],
                        "Type": "Temperature",
                        "Name": sensor["Name"],
                        "Value": sensor["ReadingCelsius"],
                        "Unit": "Celsius"
                    })

        elif server["type"] == "power":
            # PDU 電力データを抽出
            for power_ctrl in data.get("PowerControl", []):
                results.append({
                    "Timestamp": timestamp,
                    "Device": server["name"],
                    "Type": "Power",
                    "Name": power_ctrl["Name"],
                    "Value": power_ctrl["PowerConsumedWatts"],
                    "Unit": "Watts"
                })

        return results

    except requests.exceptions.RequestException as e:
        print(f"Error accessing {url}: {e}")
        return [{
            "Timestamp": timestamp,
            "Device": server["name"],
            "IP": server["url"],
            "Type": "Error",
            "Name": None,
            "Value": None,
            "Unit": None,

        }]


In [41]:
def fetch_and_display_data():
    """全サーバーと PDU からデータを取得し、DataFrame で表示"""
    all_data = []

    for server in SERVERS:
        print(f"\nFetching data from {server['name']} ({server['url']}) ...")
        server_data = get_redfish_data(server)
        all_data.extend(server_data)

    if not all_data:
        print("No data found.")
        return pd.DataFrame()  # 空の DataFrame を返す

    df = pd.DataFrame(all_data)
    
    # CSVに保存
    df.to_csv(CSV_FILE, index=False)
    
    print(f"\n✅ Data saved to {CSV_FILE}")
    
    return df

In [42]:
df = fetch_and_display_data()
df



Fetching data from Server1 (http://127.0.0.1:8001) ...

Fetching data from Server2 (http://127.0.0.1:8002) ...

Fetching data from PDU1 (http://127.0.0.1:9001) ...
Error accessing http://127.0.0.1:9001/redfish/v1/Chassis/0/Power: HTTPConnectionPool(host='127.0.0.1', port=9001): Read timed out. (read timeout=1)

✅ Data saved to server_pdu_data_20250210_233503.csv


,Timestamp,Device,IP,Type,Name,Value,Unit
0,2025-02-10 23:57:55,Server1,http://127.0.0.1:8001,Temperature,CPU1 Temp,111.0,Celsius
1,2025-02-10 23:57:55,Server1,http://127.0.0.1:8001,Temperature,CPU2 Temp,112.0,Celsius
2,2025-02-10 23:57:55,Server2,http://127.0.0.1:8002,Temperature,CPU1 Temp,221.0,Celsius
3,2025-02-10 23:57:55,Server2,http://127.0.0.1:8002,Temperature,CPU2 Temp,222.0,Celsius
4,2025-02-10 23:57:55,PDU1,http://127.0.0.1:9001,Error,None,NaN,None
